# Solar Image Classification - Basic CNN

The goal of this notebook is to classify from satellite images whether or not a home has solar panels installed. The creation of this notebook relied heavily on a [Keras blog post](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) about building neural networks with a small amount of data. To start, let's import packages:

In [1]:
import os # for navigating directories
import numpy as np
from skimage import io # used to load images as numpy arrays
from sklearn.model_selection import train_test_split # split a data set into training and testing
from scipy.misc import imresize, imsave # resize images and save as pngs
import matplotlib.pyplot as plt
import pickle

from keras.models import Model,load_model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten # all the various neural network layers
from keras.utils import np_utils # utilities for one-hot encoding of ground truth value
from keras.optimizers import RMSprop # optimization algorithm to use to search for optimal weights
from keras import regularizers # used to regularize the weights

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img # for data augmentation

import keras.backend as K # used for modifying learning rate

Using TensorFlow backend.


Next load the data as numpy arrays of the desired size. Of the image sizes I tried, I found 30 x 30 pixels to work best. Given the small amount of data, we want to minimize the number of parameters we need to estimate. Therefore, the smaller we can have the photo (and still be able to distinguish the pool), the better. In order to be sure we aren't setting the resolution too low, the first few images in each class are displayed.

In [ ]:
force_same = False

# Start by counting the number of photos in each class
panel_count = 0
for (dirpath, dirnames, filenames) in os.walk('../data/original/with panels'):
    for f in filenames:
        panel_count += 1

no_panel_count = 0
for (dirpath, dirnames, filenames) in os.walk('../data/original/without panels'):
    for f in filenames:
        no_panel_count += 1
        # for now we'll ensure the same number of pool/no pool examples to avoid biasing the model
        if (no_panel_count == panel_count) and force_same:
            break

total_count = panel_count + no_panel_count

# desired shape of the input images
shape = (64,64,3)

# numpy arrays for storing data and labels (first dimension indexes over samples)
X = np.zeros((total_count,)+shape)
y = np.zeros((total_count,))
# because of the order we are loading them, the first ones are all pool examples
y[:panel_count] = 1

index = 0
for (dirpath, dirnames, filenames) in os.walk('../data/original/with panels'):
    for f in filenames:
        # load the image as numpy array
        im = io.imread('../data/original/with panels/' + f)
        # resize image to desired shape
        im_resize = imresize(im,shape)
        # store in data matrix
        X[index,:,:,:] = im_resize
        index += 1
        if index < 4:
            plt.figure(figsize=(5,5))
            plt.imshow(im_resize)
            plt.title(f)
            plt.show()

for (dirpath, dirnames, filenames) in os.walk('../data/original/without panels'):
    for f in filenames:
        im = io.imread('../data/original/without panels/' + f)
        im_resize = imresize(im,shape)
        X[index,:,:,:] = im_resize
        index += 1
        if index - panel_count < 4:
            plt.figure(figsize=(5,5))
            plt.imshow(im_resize)
            plt.title(f)
            plt.show()
        if index == total_count:
            break
        
print("%i Panel Examples"%panel_count)
print("%i No Panel Examples"%no_panel_count)

Now that the complete data and labels are loaded we can split them into testing and training sets, store important values, and prepare the data for use by Keras.

In [5]:
# train on 2/3 and test on 1/3 of data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=4)

# store the details
num_train, height, width, depth = X_train.shape
num_test = X_test.shape[0]

# ensure input data is correct type
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')

#normalize input data
X_train /= np.max(X_train)
X_test /= np.max(X_test)

Next we specify all the parameters used to create the network.

In [11]:
model_num = 1 # can set this for saving models
batch_size = 5 # in each iteration, we consider 10 training examples at once
kernel_size = 12 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 128 # we will initially have 8 kernels per conv. layer...
conv_depth_2 = 256 # ...switching to 16 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 32 neurons
conv_reg = 0.00001 # amount of regularization to perform in convolutional layers (l2)
hidden_reg = 0.00001 # amount of regularization to perform in the hidden layer (l2)
learning_rate = 0.00001 # learning rate for optimization algorithm

Now compile the model:

In [12]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size),
                       kernel_regularizer=regularizers.l2(conv_reg),
                       padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size),
                       kernel_regularizer=regularizers.l2(conv_reg),
                       padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size),
                       kernel_regularizer=regularizers.l2(conv_reg),
                       padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size),
                       kernel_regularizer=regularizers.l2(conv_reg),
                       padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden1 = Dense(hidden_size, activation='relu',kernel_regularizer=regularizers.l2(hidden_reg))(flat)
drop_3 = Dropout(drop_prob_2)(hidden1)
out = Dense(1, activation='sigmoid')(drop_3)

# To define a model, just specify its input and output layers
model = Model(inputs=inp, outputs=out) 

# In order to specify learning rate, need to create an optimizer
rms_prop = RMSprop(lr=learning_rate)

model.compile(loss='binary_crossentropy',
              optimizer=rms_prop,
              metrics=['accuracy'])

Before training the model, increase the number of samples by augmenting our training set. First, create the generator:

In [13]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rotation_range=15.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None)

Now generate the augmented data and combine with the original data.

In [14]:
augmented_data = np.zeros((0,) + shape)
augmented_labels = np.asarray([])

# set the number of times to augment each image
factor_increase = 5
i = 0
for batch_data,batch_labels in datagen.flow(X_train, y_train, batch_size=X_train.shape[0]):
    augmented_data = np.vstack((augmented_data,batch_data))
    augmented_labels = np.append(augmented_labels,batch_labels)
    if i == factor_increase:
        break
    i += 1
    
    
X_aug_train = np.vstack((X_train,augmented_data))
y_aug_train = np.append(y_train,augmented_labels)

Now train the model. I've found it easiest to just run one epoch at a time because running on verbose can sometimes freeze the notebook but running not verbose makes it hard to know the progress. This way you can print results after some epochs, save the model after some epochs, and update the learning rate after some epochs.

In [ ]:
for i in range(400):
    fit_aug_results = model.fit(X_aug_train, y_aug_train, # Train the model using the training set...
              batch_size=3, epochs=1,
              verbose=0, validation_split=0.1,class_weight={0:1,1:2}) # ...holding out 10% of the data for validation
    if(i%10==0):
        print('*****Epoch %i*****'%i)
        print('Training Loss: %f'%fit_aug_results.history['loss'][0])
        print('Validation Loss: %f'%fit_aug_results.history['val_loss'][0])
        print('Training Accuracy: %f'%fit_aug_results.history['acc'][0])
        print('Validation Accuracy: %f'%fit_aug_results.history['val_acc'][0])
    if(i%20==0):
        print('------Saving Epoch %i------'%i)
        model.save('../models/solar_model_%i_epoch%i.h5'%(model_num,i))
    if(i%200==0):
        K.set_value(model.optimizer.lr,0.000001)
    if(i%300==0):
        K.set_value(model.optimizer.lr,0.0000005)

Next, load the model you want to analyze and check out the performance. Note that the portion below here can be run in a different notebook if the above cell is still running but has saved some models already. First let's look at the summary of results from the test set.

In [ ]:
model = load_model('../models/solar_model_1_epoch200.h5')

# predict based on test set
y_hat = model.predict(X_test,verbose=1)
# convert probabilities to True/False estimates
labels = (y_hat > 0.5)
# convert True/False to 1/0
y_hat = [label[0] for label in labels.astype(int)]

print("Test Accuracy: %f"%(float(sum(y_hat == y_test))/len(y_hat)))
print("Chance Accuracy: %f"%(1-float(sum(y_test))/len(y_hat)))
print("Solar Examples:%i"%sum(y_test == 1))
print("Misses:%i"%sum(y_hat - y_test == -1))
print("No Solar Examples:%i"%sum(y_test == 0))
print("False Alarms:%i"%sum(y_hat - y_test == 1))

Now let's loop through each test sample and look at the probability it assigned. If the classification was incorrect, display the image so we can get an idea of what sort of images cause errors. If the classification is correct with above 95% certainty, then save that sample so we can look at how it passes through the network to see what kind of features are being generated.

In [ ]:
# keep track of samples to look more closely at
X_look = np.zeros_like(X_test)
y_look = np.zeros_like(y_test)
j = 0

for i in range(X_test.shape[0]):
    x_i = X_test[i,:,:,:]
    y_i = model.predict(np.expand_dims(x_i,axis=0))[0]
    label_i = int(y_i>0.5)
    if np.abs(y_i - y_test[i]) < 0.05:
        X_look[j,:,:,:] = X_test[i,:,:,:]
        y_look[j] = y_test[i]
        j += 1
    if label_i == y_test[i]:
        outcome = 'Success'
    else:
        outcome = 'Failure'
    print('%f - %s'%(y_i,outcome))
    if y_test[i] != label_i:
        plt.figure(figsize=(5,5))
        plt.imshow(x_i)
        plt.title('P(panel)=%f - %s'%(y_i,outcome))
        plt.show()

Next we will visualize the network by creating intermediate layer models whose outputs are the outputs of the convolutional layers.

In [ ]:
# Create two intermediate layer models models
intermediate_layer_model1 = Model(inputs=model.input,
                                 outputs=model.layers[1].output)

intermediate_layer_model2 = Model(inputs=model.input,
                                 outputs=model.layers[5].output)

# for the first ten samples in our 95% confidence group
for index in range(10):
    # display the image in question
    im_resize = X_look[index,:,:,:]
    plt.figure(figsize=(4,4))
    plt.imshow(im_resize)
    plt.title(f)

    # calling predict on the intermediate model gives the filter outputs
    out1 = intermediate_layer_model1.predict(np.expand_dims(im_resize,axis=0))   
    fig,axarr = plt.subplots(6,6,figsize=(10,10))
    plt.title('layer 2 filter outputs')
    axarr = axarr.reshape(-1)
    for i in range(36):
        filt = axarr[i].imshow(np.squeeze(out1,axis=0)[:,:,i])
        axarr[i].set_title('max value: %5.2f'%(filt.get_clim()[1]))                                   
        axarr[i].axis('off')
    plt.tight_layout()
    plt.show()

    out2 = intermediate_layer_model2.predict(np.expand_dims(im_resize,axis=0))
    fig,axarr = plt.subplots(6,6,figsize=(10,10))
    plt.title('layer 6 filter outputs')
    axarr = axarr.reshape(-1)
    for i in range(36):
        filt = axarr[i].imshow(np.squeeze(out2,axis=0)[:,:,i])
        axarr[i].set_title('max value: %5.2f'%(filt.get_clim()[1]))                                   
        axarr[i].axis('off')
    plt.tight_layout()
    plt.show()